In [16]:
# Ising model monte carlo simulation

from numpy import ones, exp, arange, array
from random import randint, random
from matplotlib.pyplot import plot, xlabel, ylabel, figure, subplot, show, title, subplots_adjust
from multiprocessing import Pool
from ipywidgets import interact_manual, IntText, FloatText, Dropdown, VBox, HBox, Button, Output, Checkbox
from IPython.display import display, HTML
from functools import partial

# Constants
J = 1
kB = 1
equ = 5e3

# Visual interface
Tm = FloatText(description=r'$T_m$')
TM = FloatText(value=5.0, description=r'$T_M$')
dt = FloatText(value=0.2, description=r'$dT$')
Bf = FloatText(value=0.0, description=r'$B_{ext}$')
Steps_ = IntText(value=500, description='Steps')
Size_ = Dropdown(options=[16, 24, 32, 40], description=r'$N$')
M_ = Checkbox(description=r'$M$', value=True)
E_ = Checkbox(description=r'$E$', value=True)
Chi_ = Checkbox(description=r'$\chi$', value=True)
Cv_ = Checkbox(description=r'$C_v$', value=True)
button = Button(description='Run Simulation')
out = Output(layout={'border': '1px solid black'})

with out:
    display(HBox([VBox([Tm, TM, dt, Steps_, Size_, Bf, button]), VBox([M_, E_, Cv_, Chi_])]))

def mcsim(T, measurements, size, B):
    # Ising model Monte Carlo simulation
    global J, kB, equ
    S = ones((size, size))       # Initial lattice, all spin up
    beta = 1 / kB / T
    N = size ** 2       # Number of lattice points
    M = N       # Initial magnetization
    E = -2 * J * N - B * N       # Inital energy
    exp_EE = 0
    exp_MM = 0
    M_exp = 0
    E_exp = 0
    z = 0       # Counter to scale data
    steps = int(equ + measurements * N)       # Calculate steps to take desired number of measurements
    for k in range(steps):
        if k > equ and k % N == 0:
            # Take a measurement once per sweep after the equilibration time
            M_exp += abs(M)
            E_exp += E
            exp_EE += E ** 2
            exp_MM += M ** 2
            z += 1
        # Pick a random latice point and handle periodic boundary conditions
        i = randint(0, size - 1)
        j = randint(0, size - 1)
        if i == size - 1:
            down = 0
        else:
            down = i + 1
        if i == 0:
            up = size - 1
        else:
            up = i - 1
        if j == size - 1:
            right = 0
        else:
            right = j + 1
        if j == 0:
            left = size - 1
        else:
            left = j - 1
        # Calculate change in energy
        deltaE = 2 * S[i, j] * J * (S[up, j] + S[down, j] + S[i, left]
                      + S[i, right]) + 2 * B * S[i, j]
        # Decide whether to accept the spin
        if random() < exp(-beta * deltaE):
            S[i, j] *= -1
            M += 2 * S[i, j]
            E += deltaE
    # Scale data to return
    M_exp /= z
    E_exp /= z
    exp_MM /= z
    exp_EE /= z
    
    # Calculate magnetic susceptibility and heat capacity
    chi = exp_MM - (M_exp ** 2)
    Cv = exp_EE - (E_exp ** 2)
    return [E_exp, M_exp, chi, Cv]


def terminal(switch):
    Tmin = Tm.value
    Tmax = TM.value
    Tstep = dt.value
    Size = Size_.value
    Steps = Steps_.value
    Mstate = M_.value
    Estate = E_.value
    Cvstate = Cv_.value
    Chistate = Chi_.value
    field = Bf.value
    if Tmin == 0:
        temps = arange(Tmin + Tstep, Tmax + Tstep, Tstep)
    else:
        temps = arange(Tmin, Tmax + Tstep, Tstep)
    cores = Pool()
    sim = partial(mcsim, measurements = Steps, size = Size, B = field)
    results = cores.map(sim, temps)
    cores.close()
    cores.join()
    data = array(results)
    E_pts = data[:, 0]
    M_pts = data[:, 1]
    chi_pts = data[:, 2]
    Cv_pts = data[:, 3]
    # Plot the data
    fig = figure()
    if Mstate == True:
        Max = fig.add_subplot(221)
        Max.set_title(r'$M(T)$')
        Mcurve = Max.plot(temps, M_pts, 'b')
    if Estate == True:
        Eax = fig.add_subplot(222)
        Eax.set_title(r'$E(T)$')
        Ecurve = Eax.plot(temps, E_pts, 'g')
    if Cvstate == True:
        Cvax = fig.add_subplot(223)
        Cvax.set_title(r'$Cv(T)$')
        Cvcurve = Cvax.plot(temps, Cv_pts, 'r')
    if Chistate == True:
        Chiax = fig.add_subplot(224)
        Chiax.set_title(r'$\chi(T)$')
        Chicurve = Chiax.plot(temps, chi_pts, 'c')
    
    subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.35, wspace=0.35)
    
    show()
    
    
# Control
button.on_click(terminal)

# Display
out

Output(layout=Layout(border='1px solid black'))

In [17]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')